# 数据预处理
:label:`sec_pandas`

为了能用深度学习来解决现实世界的问题，我们经常从预处理原始数据开始，
而不是从那些准备好的张量格式数据开始。
在Python中常用的数据分析工具中，我们通常使用`pandas`软件包。
像庞大的Python生态系统中的许多其他扩展包一样，`pandas`可以与张量兼容。
本节我们将简要介绍使用`pandas`预处理原始数据，并将原始数据转换为张量格式的步骤。
我们将在后面的章节中介绍更多的数据预处理技术。

## 读取数据集

举一个例子，我们首先(**创建一个人工数据集，并存储在CSV（逗号分隔值）文件**)
`../data/house_tiny.csv`中。
以其他格式存储的数据也可以通过类似的方式进行处理。
下面我们将数据集按行写入CSV文件中。


In [2]:
import os

os.makedirs(os.path.join('..', 'data'), exist_ok=True)
data_file = os.path.join('..', 'data', 'house_tiny.csv')
with open(data_file, 'w') as f:
    f.write('NumRooms,Alley,Price\n')  # 列名
    f.write('NA,Pave,127500\n')  # 每行表示一个数据样本
    f.write('2,NA,106000\n')
    f.write('4,NA,178100\n')
    f.write('NA,NA,140000\n')

要[**从创建的CSV文件中加载原始数据集**]，我们导入`pandas`包并调用`read_csv`函数。该数据集有四行三列。其中每行描述了房间数量（“NumRooms”）、巷子类型（“Alley”）和房屋价格（“Price”）。


In [3]:
# 如果没有安装pandas，只需取消对以下行的注释来安装pandas
# !pip install pandas
import pandas as pd

data = pd.read_csv(data_file)
print(data)

   NumRooms Alley   Price
0       NaN  Pave  127500
1       2.0   NaN  106000
2       4.0   NaN  178100
3       NaN   NaN  140000


## 处理缺失值

注意，“NaN”项代表缺失值。
[**为了处理缺失的数据，典型的方法包括*插值法*和*删除法*，**]
其中插值法用一个替代值弥补缺失值，而删除法则直接忽略缺失值。
在(**这里，我们将考虑插值法**)。

通过位置索引`iloc`，我们将`data`分成`inputs`和`outputs`，
其中前者为`data`的前两列，而后者为`data`的最后一列。
对于`inputs`中缺少的数值，我们用同一列的均值替换“NaN”项。


In [20]:
inputs, outputs = data.iloc[:, 0:2], data.iloc[:, 2]
inputs = inputs.fillna(inputs.mean())
print(inputs)

TypeError: can only concatenate str (not "int") to str

[**对于`inputs`中的类别值或离散值，我们将“NaN”视为一个类别。**]
由于“巷子类型”（“Alley”）列只接受两种类型的类别值“Pave”和“NaN”，
`pandas`可以自动将此列转换为两列“Alley_Pave”和“Alley_nan”。
巷子类型为“Pave”的行会将“Alley_Pave”的值设置为1，“Alley_nan”的值设置为0。
缺少巷子类型的行会将“Alley_Pave”和“Alley_nan”分别设置为0和1。


In [18]:
inputs = pd.get_dummies(inputs, dummy_na=True)
print(inputs)

   NumRooms  Alley_Pave  Alley_nan
0       NaN        True      False
1       2.0       False       True
2       4.0       False       True
3       NaN       False       True


## 转换为张量格式

[**现在`inputs`和`outputs`中的所有条目都是数值类型，它们可以转换为张量格式。**]
当数据采用张量格式后，可以通过在 :numref:`sec_ndarray`中引入的那些张量函数来进一步操作。


In [ ]:
import torch

X, y = torch.tensor(inputs.values), torch.tensor(outputs.values)
X, y

(tensor([[3., 1., 0.],
         [2., 0., 1.],
         [4., 0., 1.],
         [3., 0., 1.]], dtype=torch.float64),
 tensor([127500, 106000, 178100, 140000]))

## 小结

* `pandas`软件包是Python中常用的数据分析工具中，`pandas`可以与张量兼容。
* 用`pandas`处理缺失的数据时，我们可根据情况选择用插值法和删除法。

## 练习

创建包含更多行和列的原始数据集。

1. 删除缺失值最多的列。
2. 将预处理后的数据集转换为张量格式。


[Discussions](https://discuss.d2l.ai/t/1750)


# 問題1: 删除缺失值最多的列。

In [14]:
import pandas as pd
import numpy as np

# 建立一個100行10列的資料集
raw_data = pd.DataFrame(np.random.randn(10,10)) 

# 在2,4,7列中加入缺失值
raw_data.iloc[:8,2] = np.nan
raw_data.iloc[5:8,4] = np.nan 
raw_data.iloc[8:,7] = np.nan

# 統計每列的缺失值個數
missing_count = raw_data.isnull().sum()
print(raw_data)

          0         1         2         3         4         5         6  \
0  0.089604  0.737606       NaN -0.813122  0.016740  1.041876 -0.554007   
1 -0.951437  0.079575       NaN -0.380885  0.594517 -0.273405  0.243241   
2 -0.489010 -0.942634       NaN -0.272143 -0.480395 -0.593877 -1.161195   
3 -0.278576 -0.414861       NaN  0.007159 -1.404076 -0.207324  1.593478   
4 -1.836826  0.180240       NaN  0.012615  1.471118  0.141673  0.524310   
5  0.380813  2.671936       NaN  1.601793       NaN  1.458587 -0.579347   
6 -0.559325  0.581682       NaN  1.558354       NaN  1.888063 -0.527195   
7 -1.323071 -0.917318       NaN -3.315996       NaN -0.288532  1.529429   
8 -2.114200 -0.807402  1.741899  0.006645  0.649047  0.102802 -1.428705   
9  0.208017  1.124217  0.640780 -0.011292 -0.674760 -1.324886  0.536741   

          7         8         9  
0 -0.294467 -1.231803  0.205720  
1 -1.332085  1.222034 -0.161186  
2  0.577695 -0.220094 -1.133463  
3  0.393448 -3.273221  0.782935  
4  0

In [15]:
# 取得缺失值最多的那一列
most_missing_col = missing_count[missing_count==missing_count.max()].index[0] 
print(most_missing_col)

2


In [12]:
# 删除缺失值最多的那一列
raw_data.drop(most_missing_col,axis=1,inplace=True)

print(raw_data)

          0         1         3         4         5         6         7  \
0  0.848236 -0.604565 -2.125199 -0.561131  0.666404 -0.357414  0.719475   
1 -0.763982  1.264226  1.527255  0.405355  1.365731  2.482153  0.418386   
2 -0.316437 -0.597054 -1.050982 -0.301949 -0.793415 -0.061573 -0.840920   
3  0.358027 -0.045710  0.806148  0.125088  0.068384  0.827972 -1.436924   
4 -0.791207 -0.439528 -0.679347 -1.923081 -0.041273 -0.643833 -2.245317   
5 -0.047117  0.286936 -0.282733       NaN -0.149701  0.274552  0.317985   
6 -0.031458 -0.537391  0.708874  0.147732 -0.103467 -2.248026  0.137485   
7 -1.047800  1.035614 -0.647511  1.889346  0.223507 -0.597081  0.184199   
8  0.172889 -0.822085  0.037135  1.085364 -0.903427  1.255189       NaN   
9 -0.235407  0.925761 -0.870109 -0.888770 -0.198729 -1.198686       NaN   

          8         9  
0  0.097712 -0.730359  
1  1.988654  1.372115  
2  2.921052 -2.159848  
3  0.650842 -0.570485  
4 -2.684897  0.596402  
5 -0.516841  1.087877  
6  0.5

# 問題2 :